In [ ]:
from jaf.core.llm import AzureGPTLLM 
import os 

AZURE_GPT_DEPLOYMENT_NAME=os.getenv("AZURE_GPT_DEPLOYMENT_NAME")
AZURE_OAI_BASE_URL=os.getenv("AZURE_OAI_BASE_URL")
AZURE_OAI_API_VERSION= os.getenv("AZURE_OAI_API_VERSION")
AZURE_OAI_API_KEY= os.getenv("AZURE_OAI_API_KEY")
llm = AzureGPTLLM(
    deployment_name=AZURE_GPT_DEPLOYMENT_NAME,
    api_base=AZURE_OAI_BASE_URL,
    api_version=AZURE_OAI_API_VERSION,
    api_key=AZURE_OAI_API_KEY
)

In [57]:
from pydantic import BaseModel

class Data(BaseModel):
   moduleName: str
   addedFunctions: list = []
   modifiedFunctions: list = []
   deletedFunctions: list = []
   addedTypes: list = []
   modifiedTypes: list = []
   deletedTypes: list = []
   addedExternals: list = []
   modifiedExternals: list = []
   deletedExternals: list = []

In [58]:
import json

with open('/Users/sakthi.n/Documents/Opensource/cada-rescript/detailed_changes.json', 'r') as file:
    data = json.load(file)

result = []
for d in data:
    result.append(Data.model_validate(d))

In [96]:
systemPrompt = """ 
You are a QA automation assistant. Given code changes to frontend components or functions (added or modified), generate structured test case objects in JSON format.

Each test case must contain:

testCaseId: Unique identifier (e.g., TC001, TC002)

description: One-line summary of what the test verifies

stepsToReproduce: A list of precise UI actions (e.g., "Click button", "Enter value in input")

expectedResult: What the correct outcome should be

⚠️ Steps must be realistic, like how a manual tester would execute them.
❌ Avoid vague steps like "Run function" — describe the UI interaction instead.

Output a JSON array of such test cases.

If there are no test cases (e.g., only deletions), return an empty list [].

Eg Format:
[
  {
    "testCaseId": "TC001",
    "description": "User can successfully search when input is valid",
    "stepsToReproduce": [
      "Navigate to the Search page",
      "Enter 'apple' in the search input box",
      "Click the 'Search' button"
    ],
    "expectedResult": "Search results are displayed for 'apple'"
  },
  {
    "testCaseId": "TC002",
    "description": "Search button does nothing if query is too short",
    "stepsToReproduce": [
      "Navigate to the Search page",
      "Enter 'ap' in the search input box",
      "Click the 'Search' button"
    ],
    "expectedResult": "No search is performed; no request sent"
  }
]
"""

userPrompt = """
Here are the frontend code changes in this pull request.
Please generate test cases in JSON format, following this structure:
[
  {{
    "testCaseId": "TC001",
    "description": "What the test is verifying",
    "stepsToReproduce": [
      "Step 1",
      "Step 2",
      "..."
    ],
    "expectedResult": "Expected behavior"
  }}
]
Be realistic and user-centric in the steps — describe UI actions like clicking, typing, navigation, etc.
Generate test cases only for added or modified functions or components.
Ignore deleted functions unless they affect user-facing behavior.

Added Functions/Components:
{added}

Modified Functions/Components:
{modified}

Deleted Functions/Components:
{deleted}

Based on the above things 
Return the test cases as a JSON array.
Do not include any extra commentary or explanation — just valid JSON.
"""

In [98]:
from jaf.types import Query 

q = Query()
q.prompt = userPrompt.format(added=result[0].addedFunctions,modified=result[0].modifiedFunctions,deleted=result[0].deletedFunctions)
q.system_prompt = systemPrompt 

# print(q.prompt)
llm(q)

print(q.response)

```json
[
  {
    "testCaseId": "TC001",
    "description": "User can enter encryption logic description",
    "stepsToReproduce": [
      "Navigate to the form page",
      "Locate the input field labeled 'What encryption and decryption logic do you support?'",
      "Enter 'AES-256 encryption' in the input field"
    ],
    "expectedResult": "The input field accepts the text 'AES-256 encryption'"
  },
  {
    "testCaseId": "TC002",
    "description": "User can enter sample encrypted and decrypted payload",
    "stepsToReproduce": [
      "Navigate to the form page",
      "Locate the multi-line input field labeled 'Please share a sample encrypted and decrypted payload'",
      "Enter 'Sample payload data' in the input field"
    ],
    "expectedResult": "The multi-line input field accepts the text 'Sample payload data'"
  },
  {
    "testCaseId": "TC003",
    "description": "User can enter encryption keys",
    "stepsToReproduce": [
      "Navigate to the form page",
      "Locate th